<a href="https://colab.research.google.com/github/kimdesok/whole-slide-cnn/blob/main/Calc_GPU_memory_requirement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Author: Dr. Sreenivas Bhattiprolu 
Calculation of memory required to store a batch of images and features 
in a deep learning model. In addition, we also add the memory required
to store trainable and non trainable parameters.
Remember that you need at least this much memory but in reality you have 
other overheads. 
If you do not have this much bare minimum memory then no point in trying to train
your model. Consider working with smaller images or batch sizes. 
"""
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import backend as K
import numpy as np

In [2]:
def get_model_memory_usage(batch_size, model):
    
    features_mem = 0 # Initialize memory for features. 
    float_bytes = 4.0 #Multiplication factor as all values we store would be float32.
    
    for layer in model.layers:

        out_shape = layer.output_shape
        
        if type(out_shape) is list:   #e.g. input layer which is a list
            out_shape = out_shape[0]
        else:
            out_shape = [out_shape[1], out_shape[2], out_shape[3]]
            
        #Multiply all shapes to get the total number per layer.    
        single_layer_mem = 1 
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        
        single_layer_mem_float = single_layer_mem * float_bytes #Multiply by 4 bytes (float)
        single_layer_mem_MB = single_layer_mem_float/(1024**2)  #Convert to MB
        
        #print("Memory for", out_shape, " layer in MB is:", single_layer_mem_MB)
        features_mem += single_layer_mem_MB  #Add to total feature memory count

# Calculate Parameter memory
    trainable_wts = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_wts = np.sum([K.count_params(p) for p in model.non_trainable_weights])
    parameter_mem_MB = ((trainable_wts + non_trainable_wts) * float_bytes)/(1024**2)
    print("----------------------------------------")
    print("Memory for features in GB is: %.2f" %(features_mem*batch_size/1024))
    print("Memory for parameters in GB is: %.2f" %(parameter_mem_MB/1024))

    total_memory_MB = (batch_size * features_mem) + parameter_mem_MB  #Same number of parameters. independent of batch size

    total_memory_GB = total_memory_MB/1024
    
    return total_memory_GB

In [5]:
#Let us import VGG16 model.
input_image_shape = (10000, 10000, 3)
#model = VGG16(include_top=False, input_shape=input_image_shape)
model = ResNet50(include_top=False, input_shape=input_image_shape)
#print(model.summary())
BATCH_SIZE = 1

##############################################################
batch_size = BATCH_SIZE
mem_for_my_model = get_model_memory_usage(batch_size, model)

print("----------------------------------------")
print("Minimum memory required to work with this model is: %.2f" %mem_for_my_model, "GB")


###############################################################

----------------------------------------
Memory for features in GB is: 275.35
Memory for parameters in GB is: 0.09
----------------------------------------
Minimum memory required to work with this model is: 275.44 GB


In [ ]:
# Expense estimation in case purchasing more RAM
my_mem = 410; time_to_run = 150; won_per_hour_per_GB = 1.6
expense = (mem_for_my_model - my_mem);print("Lacking this much memory, %d GB " % int(expense))
expense = expense * time_to_run * won_per_hour_per_GB
print("Needs this much money, %d won per job" % int(expense))

Lacking this much memory, 922 GB 
Needs this much money, 221434 won per job
